In [ ]:
import pandas as pd
d = pd.read_csv('Youtube Spam Classification/Youtube01-Psy.csv')

In [ ]:
d.tail()

In [ ]:
len(d.query('CLASS == 1'))

In [ ]:
len(d.query('CLASS == 0'))

In [ ]:
len(d)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [ ]:
dvec = vectorizer.fit_transform(d['CONTENT'])

In [ ]:
print(dvec)

In [ ]:
analyze = vectorizer.build_analyzer()

In [ ]:
print(d['CONTENT'][349])
analyze(d['CONTENT'][349])

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
dshuf = d.sample(frac=1)

In [ ]:
d_train = dshuf[:300]
d_test = dshuf[300:]
d_train_att = vectorizer.fit_transform(d_train['CONTENT'])
d_test_att = vectorizer.transform(d_test['CONTENT'])
d_train_label = d_train['CLASS']
d_test_label = d_test['CLASS']

In [ ]:
print(d_train_att[0])

In [ ]:
print(d_train_label)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=80)

In [ ]:
clf.fit(d_train_att, d_train_label)

In [ ]:
clf.score(d_test_att, d_test_label)

In [ ]:
from sklearn.metrics import confusion_matrix
pred_labels = clf.predict(d_test_att)
confusion_matrix(d_test_label, pred_labels)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, d_train_att, d_train_label, cv=5)
print('Accuracy: %0.2f (+/- %0.2f)' %(scores.mean(), scores.std()*2))

In [ ]:
d = pd.concat([pd.read_csv('Youtube Spam Classification/Youtube01-Psy.csv'),
              pd.read_csv('Youtube Spam Classification/Youtube02-KatyPerry.csv'),
              pd.read_csv('Youtube Spam Classification/Youtube03-LMFAO.csv'),
              pd.read_csv('Youtube Spam Classification/Youtube04-Eminem.csv'),
              pd.read_csv('Youtube Spam Classification/Youtube05-Shakira.csv')])

In [ ]:
d

In [ ]:
len(d)

In [ ]:
len(d.query('CLASS == 1'))

In [ ]:
len(d.query('CLASS == 0'))

In [ ]:
dshuf = d.sample(frac=1)
d_content = dshuf['CONTENT']
d_label = dshuf['CLASS']

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
pipeline = Pipeline([
    ('bag-of-words', CountVectorizer()),
    ('random-forest', RandomForestClassifier())
])
pipeline

In [ ]:
pipeline.fit(d_content[:1500], d_label[:1500])

In [ ]:
pipeline.score(d_content[1500:], d_label[1500:])

In [ ]:
pipeline.predict(['it is a very good video!'])

In [ ]:
pipeline.predict(['it is a very good video! (Subscribee to my cahnel)'])

In [ ]:
scores = cross_val_score(pipeline, d_content, d_label, cv=5)
print('Accuracy: %0.2f (+/- %0.2f)' %(scores.mean(), scores.std()*2))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
pipeline2 = make_pipeline(CountVectorizer(),
                         TfidfTransformer(norm=None),
                         RandomForestClassifier())

In [ ]:
scores = cross_val_score(pipeline2, d_content, d_label, cv=5)
print('Accuracy: %0.2f (+/- %0.2f)' %(scores.mean(), scores.std()*2))

In [ ]:
pipeline2.steps

In [ ]:
parameters = { 
    'countvectorizer__max_features': (None, 1000, 2000), 
    'countvectorizer__ngram_range': ((1, 1), (1, 2)), #unigrams or bigrams 
    'countvectorizer__stop_words': ('english', None), 
    'tfidftransformer__use_idf': (True, False), # effectively turn on/off tfidf 
    'randomforestclassifier__n_estimators': (20, 50, 100) 
} 
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline2, parameters, n_jobs=-1, verbose=1)

In [ ]:
grid_search.fit(d_content, d_label)

In [ ]:
print("Best score: %0.3f" % grid_search.best_score_) 
print("Best parameters set:") 
best_parameters = grid_search.best_estimator_.get_params() 
for param_name in sorted(parameters.keys()): 
    print("\t%s: %r" % (param_name, best_parameters[param_name]))